# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import random
import datetime

# Создадим датафрэйм с переводами

## Создадим id отправителей

In [3]:
id_sender = []
for i in range(0, 100000):
    id_sender.append(random.randint(1000, 10000))

## Создадим id получателей

In [4]:
id_receiver = []
for i in range(0, 100000):
    id_receiver.append(random.randint(1000, 10000))

## Создадим суммы переводов

In [12]:
amount = []
for i in range(100000):
    res = random.uniform(100.5, 1000000)
    amount.append(round(res,2))

## Создадим период операций

In [17]:
dates = []
for i in range(100000):
    year = random.randint(2023, 2024)
    month = random.randint(1, 12)
    day = random.randint(1, 31)
    try:
        date = datetime.date(year, month, day)
    except ValueError:
        day = 28 #учтем, что не во всех месяцах есть 31 число, а в феврале только 28 дней (поэтому указываем min = 28)
        date = datetime.date(year, month, day)
    dates.append(date)

## Итоговый датафрэйм с операциями

In [18]:
list_tuples = list(zip(id_sender, id_receiver, dates, amount)) 

In [19]:
df = pd.DataFrame(list_tuples, columns = ['id_sender', 'id_receiver', 'dates', 'amount_new'])

# Создадим датафрйэм с отправителями в интересующий нас день (например, выдачи денежных средств)

## Оставим только id_sender из предыдущего датафрэйма

In [21]:
df_first = df[['id_sender']].drop_duplicates().reset_index().drop(columns = 'index')

## Создадим даты выдачи средств

In [22]:
dates_cred = []
for i in range(len(df_first)):
    year = random.randint(2023, 2024)
    month = random.randint(1, 12)
    day = random.randint(1, 31)
    try:
        date = datetime.date(year, month, day)
    except ValueError:
        day = 28
        date = datetime.date(year, month, day)
    dates_cred.append(date)

## Итоговый файл

In [23]:
df_first['date_cred'] = dates_cred

# Итак, у нас есть файл с переводами и изначальный файл с отправителями с интрсующим нас днем. Произведем очистку фин связей
## Под очисткой фин связи будем понимать следующее: пусть интерсующий нас  день - это Т. Отправитель мог направить мошенникам деньги за период Т-1 и Т+7. При этом, чтобы вывести именно мошенников, необходимо в этом периоде убрать тех получателей, которым отправитель переводил до этого периода и после (вряд ли они были знакомы и вряд ли подружились после обмана)

### Вычислим период между операцией и нужной нам датой

In [34]:
mer = df.merge(df_first, on = 'id_sender', how = 'left')

In [35]:
mer['diff_time'] = mer['dates'] - mer['date_cred']

In [36]:
mer['diff_time'] = mer['diff_time'].astype(str)
mer['diff_time'] = mer['diff_time'].apply(lambda x: x.split(' ')[0])
# если данные станут '0:00:00', то заменим их на ноль
mer['diff_time'] = mer['diff_time'].apply(lambda x: x.replace('0:00:00', '0'))
mer['diff_time'] = mer['diff_time'].astype(int)

In [37]:
mer

,id_sender,id_receiver,dates,amount_new,date_cred,diff_time
0,3684,4676,2023-03-14,253006.18,2024-07-28,-502
1,9825,8146,2023-12-18,295802.95,2024-07-10,-205
2,1156,5379,2023-03-09,635643.48,2023-07-30,-143
3,1263,3873,2023-09-09,511433.55,2024-12-16,-464
4,1702,9413,2023-07-11,165855.39,2023-05-29,43
...,...,...,...,...,...,...
99995,9706,3523,2024-02-28,646926.28,2024-11-22,-268
99996,9127,5575,2024-06-27,143797.24,2023-06-21,372
99997,5813,2807,2024-12-31,868164.79,2024-08-06,147
99998,2296,8539,2024-08-27,735816.27,2023-01-16,589


## Оставим нтересный нам период (Т-1 и Т+7)

In [46]:
mer_intresting = mer[(mer['diff_time'] >= -1) & (mer['diff_time'] <= 7)].reset_index().drop(columns = 'index')

In [47]:
mer_intresting # 1258 операций

,id_sender,id_receiver,dates,amount_new,date_cred,diff_time
0,3636,7465,2023-05-01,508517.03,2023-05-01,0
1,2410,8685,2023-03-16,605139.45,2023-03-10,6
2,9573,9594,2024-01-24,100849.20,2024-01-24,0
3,6640,2124,2023-11-21,986239.06,2023-11-22,-1
4,4963,4630,2023-12-04,977927.02,2023-12-01,3
...,...,...,...,...,...,...
1253,7488,9748,2024-01-18,782996.68,2024-01-14,4
1254,7980,9581,2024-04-24,107254.59,2024-04-17,7
1255,5389,4323,2024-11-08,995078.00,2024-11-02,6
1256,7414,4599,2024-11-28,981944.98,2024-11-27,1


## Выведем с кем была связь до данного периода

In [44]:
mer_bad_do = mer.query('diff_time < -1')
mer_bad_posle = mer.query('diff_time > 7')
mer_bad = pd.concat([mer_bad_do, mer_bad_posle]).reset_index().drop(columns = 'index')

mer_bad = mer_bad[['id_sender', 'id_receiver']].drop_duplicates().reset_index().drop(columns = 'index')

mer_bad['flag'] = 'Фин связь' # ставим флаг

In [45]:
mer_bad

,id_sender,id_receiver,flag
0,3684,4676,Фин связь
1,9825,8146,Фин связь
2,1156,5379,Фин связь
3,1263,3873,Фин связь
4,8940,4495,Фин связь
...,...,...,...
98678,4285,1642,Фин связь
98679,2185,3831,Фин связь
98680,9127,5575,Фин связь
98681,5813,2807,Фин связь


## Соеднияем данные mer_bad к mer_intresing и оставляем пометку, что флаг должен быть пустым

In [50]:
mer_intresing_chist = mer_intresting.merge(mer_bad, on = ['id_sender', 'id_receiver'] ,how = 'left')

In [51]:
mer_intresing_chist = mer_intresing_chist[mer_intresing_chist['flag'].isna()]

In [53]:
mer_intresing_chist # осталось 125 операции (с двумя ранее была связь)

,id_sender,id_receiver,dates,amount_new,date_cred,diff_time,flag
0,3636,7465,2023-05-01,508517.03,2023-05-01,0,NaN
1,2410,8685,2023-03-16,605139.45,2023-03-10,6,NaN
2,9573,9594,2024-01-24,100849.20,2024-01-24,0,NaN
3,6640,2124,2023-11-21,986239.06,2023-11-22,-1,NaN
4,4963,4630,2023-12-04,977927.02,2023-12-01,3,NaN
...,...,...,...,...,...,...,...
1253,7488,9748,2024-01-18,782996.68,2024-01-14,4,NaN
1254,7980,9581,2024-04-24,107254.59,2024-04-17,7,NaN
1255,5389,4323,2024-11-08,995078.00,2024-11-02,6,NaN
1256,7414,4599,2024-11-28,981944.98,2024-11-27,1,NaN


# В реальных же случаях связей обнаруживается куда больше. Таким образом, были выведены операции с потенциальными мошенниками. Далее, по ним можно строить граф (в следующим файле)